In [32]:
!python --version

Python 3.11.4


In [34]:
!pip install requests

In [35]:
!pip install bs4

In [36]:
!pip install pandas

In [41]:
!pip install numpy

In [42]:
!pip install datetime

  Obtaining dependency information for datetime from https://files.pythonhosted.org/packages/f3/78/8e382b8cb4346119e2e04270b6eb4a01c5ee70b47a8a0244ecdb157204f7/DateTime-5.5-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/52.6 kB ? eta -:--:--
   ---------------------------------------- 52.6/52.6 kB 1.4 MB/s eta 0:00:00


In [66]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import sqlite3
from datetime import datetime 

In [67]:
url = 'https://en.wikipedia.org/wiki/List_of_largest_banks'
db_name = 'Banks.db'
table_name = 'Largest_banks' 
table_attribs = ["Name", "MC_USD_Billion"]
csv_path_exchange = 'https://github.com/berislav-vidakovic/Portfolio/blob/main/PythonETL/exch_rate.csv'
csv_path_load = 'Largest_banks_data.csv'
log_file = 'code_log.txt'

In [68]:
#LOGGING DATA ----------------------------------------------------------
def log_progress(message): 
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(timestamp + ' : ' + message + '\n') 

In [69]:
# EXTRACT DATA -------------------------------------------------------
def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    
    df = pd.DataFrame(columns=table_attribs)
    # load the entire web page as an HTML document in python
    html_page = requests.get(url).text
    # parse the text  to enable extraction of relevant information.
    data = BeautifulSoup(html_page, 'html.parser')
    # extract the rows of the table needed 
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr') #<class 'bs4.element.ResultSet'>
    # iterate over the rows, col->dict->df
    for row in rows:
        col=row.find_all('td')
        table_data_row=[item.text.strip() for item in col]
        if len(table_data_row) > 2:
            data_dict = { table_attribs[0]: table_data_row[1],
                          table_attribs[1]: table_data_row[2], }
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df,df1], ignore_index=True)             
    
    df['MC_USD_Billion'] = df['MC_USD_Billion'].astype('float')
    log_progress('Data extraction complete. Initiating Transformation process')
    return df

In [70]:
# TRANSFORM function ---------------------------------------------------
def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''

    dfExchange = pd.read_csv(csv_path)
    dfExchange.set_index('Currency', inplace=True)
    
    eur = dfExchange.at['EUR', 'Rate']
    gbp = dfExchange.at['GBP', 'Rate']
    inr = dfExchange.at['INR', 'Rate']

    df['MC_EUR_Billion'] = round(df['MC_USD_Billion'] * eur, 2) 
    df['MC_GBP_Billion'] = round(df['MC_USD_Billion'] * gbp, 2)  
    df['MC_INR_Billion'] = round(df['MC_USD_Billion'] * inr, 2)
    
    log_progress('Data transformation complete. Initiating Loading process')    

In [71]:
# LOAD TO CSV
def load_to_csv(df, csv_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.
    '''
    df.to_csv(csv_path, index=False)
    log_progress('Data saved to CSV file')        

In [72]:
# LOAD TO DB
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    df.to_sql(table_name, sql_connection, if_exists = 'replace', index =False)
    log_progress('Data loaded to Database as a table, Executing queries')

In [73]:
def run_queries(sql_connection, query_statement):
    ''' This function runs the passed query and adds query statement
    and query results to the log file. Function returns nothing.'''
    query_output = pd.read_sql(query_statement, sql_connection)
    log_progress(f"Running query... {query_statement}")
    log_progress(f"...Query output:\n{query_output}")

In [74]:
url = 'https://en.wikipedia.org/wiki/List_of_largest_banks'
db_name = 'Banks.db'
table_name = 'Largest_banks' 
table_attribs = ["Name", "MC_USD_Billion"]
csv_path_exchange = 'https://github.com/berislav-vidakovic/Portfolio/blob/main/PythonETL/exch_rate.csv'
csv_path_load = 'Largest_banks_data.csv'
log_file = 'code_log.txt'
log_progress("Preliminaries complete. Initiating ETL process")

# EXTRACT DATA
df=extract(url, table_attribs)

# TRANSFORM DATA
#Adding "?raw=true" at the end of the GitHub URL to get the raw file link
transform(df, csv_path_exchange+"?raw=true")

#LOAD DATA
load_to_csv(df,csv_path_load)

# create and connect  process to a  database Banks.db
conn = sqlite3.connect(db_name)
log_progress("SQL Connection initiated")

load_to_db(df, conn, table_name)

run_queries(conn, "SELECT Name, MC_USD_Billion FROM Largest_banks")
run_queries(conn, "SELECT Name, MC_GBP_Billion FROM Largest_banks")
run_queries(conn, "SELECT AVG(MC_EUR_Billion) AS Average_EUR FROM Largest_banks")
run_queries(conn, "SELECT Name, MC_INR_Billion FROM Largest_banks LIMIT 5")

# close DB connection
conn.close()
log_progress("Server Connection closed")

In [75]:
df

,Name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion
0,JPMorgan Chase,491.76,457.34,393.41,40791.49
1,Bank of America,266.45,247.80,213.16,22102.03
2,Industrial and Commercial Bank of China,219.45,204.09,175.56,18203.38
3,Wells Fargo,178.74,166.23,142.99,14826.48
4,Agricultural Bank of China,175.69,163.39,140.55,14573.49
5,HDFC Bank,169.84,157.95,135.87,14088.23
6,HSBC Holdings PLC,156.13,145.20,124.90,12950.98
7,Morgan Stanley,153.05,142.34,122.44,12695.50
8,China Construction Bank,151.97,141.33,121.58,12605.91
9,Bank of China,150.39,139.86,120.31,12474.85


In [76]:
df[['Name', 'MC_GBP_Billion']]

,Name,MC_GBP_Billion
0,JPMorgan Chase,393.41
1,Bank of America,213.16
2,Industrial and Commercial Bank of China,175.56
3,Wells Fargo,142.99
4,Agricultural Bank of China,140.55
5,HDFC Bank,135.87
6,HSBC Holdings PLC,124.90
7,Morgan Stanley,122.44
8,China Construction Bank,121.58
9,Bank of China,120.31


In [77]:
df['MC_EUR_Billion'].mean()

196.55299999999997

In [78]:
df[['Name', 'MC_INR_Billion']].head(5)

,Name,MC_INR_Billion
0,JPMorgan Chase,40791.49
1,Bank of America,22102.03
2,Industrial and Commercial Bank of China,18203.38
3,Wells Fargo,14826.48
4,Agricultural Bank of China,14573.49
